## Imports:

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import matplotlib.cm as cm
from IPython. display import clear_output
import time
from mycolorpy import colorlist as mcp
import PIL
from Superstuff.matching import Matching

## Functions:

In [ ]:
def make_matching_plot_fast(image0, image1, kpts0, kpts1, mkpts0,
                            mkpts1, color, text, path=None,
                            show_keypoints=False, margin=10,
                            opencv_display=False, opencv_title='',
                            small_text=[]):
    H0, W0 = image0.shape
    H1, W1 = image1.shape
    H, W = max(H0, H1), W0 + W1 + margin

    out = 255*np.ones((H, W), np.uint8)
    out[:H0, :W0] = image0
    out[:H1, W0+margin:] = image1
    out = np.stack([out]*3, -1)

    if show_keypoints:
        kpts0, kpts1 = np.round(kpts0).astype(int), np.round(kpts1).astype(int)
        white = (255, 255, 255)
        black = (0, 0, 0)
        for x, y in kpts0:
            cv2.circle(out, (x, y), 2, black, -1, lineType=cv2.LINE_AA)
            cv2.circle(out, (x, y), 1, white, -1, lineType=cv2.LINE_AA)
        for x, y in kpts1:
            cv2.circle(out, (x + margin + W0, y), 2, black, -1,
                       lineType=cv2.LINE_AA)
            cv2.circle(out, (x + margin + W0, y), 1, white, -1,
                       lineType=cv2.LINE_AA)

    mkpts0, mkpts1 = np.round(mkpts0).astype(int), np.round(mkpts1).astype(int)
    color = (np.array(color[:, :3])*255).astype(int)[:, ::-1]
    for (x0, y0), (x1, y1), c in zip(mkpts0, mkpts1, color):
        c = c.tolist()
        cv2.line(out, (x0, y0), (x1 + margin + W0, y1),
                 color=c, thickness=1, lineType=cv2.LINE_AA)
        # display line end-points as circles
        cv2.circle(out, (x0, y0), 2, c, -1, lineType=cv2.LINE_AA)
        cv2.circle(out, (x1 + margin + W0, y1), 2, c, -1,
                   lineType=cv2.LINE_AA)

    # Scale factor for consistent visualization across scales.
    sc = min(H / 640., 2.0)

    # Big text.
    Ht = int(30 * sc)  # text height
    txt_color_fg = (255, 255, 255)
    txt_color_bg = (0, 0, 0)
    for i, t in enumerate(text):
        cv2.putText(out, t, (int(8*sc), Ht*(i+1)), cv2.FONT_HERSHEY_DUPLEX,
                    1.0*sc, txt_color_bg, 2, cv2.LINE_AA)
        cv2.putText(out, t, (int(8*sc), Ht*(i+1)), cv2.FONT_HERSHEY_DUPLEX,
                    1.0*sc, txt_color_fg, 1, cv2.LINE_AA)

    # Small text.
    Ht = int(18 * sc)  # text height
    for i, t in enumerate(reversed(small_text)):
        cv2.putText(out, t, (int(8*sc), int(H-Ht*(i+.6))), cv2.FONT_HERSHEY_DUPLEX,
                    0.5*sc, txt_color_bg, 2, cv2.LINE_AA)
        cv2.putText(out, t, (int(8*sc), int(H-Ht*(i+.6))), cv2.FONT_HERSHEY_DUPLEX,
                    0.5*sc, txt_color_fg, 1, cv2.LINE_AA)

    if path is not None:
        cv2.imwrite(str(path), out)

    if opencv_display:
        cv2.imshow(opencv_title, out)
        cv2.waitKey(1)
    
    fig, ax = plt.subplots(figsize=(20,10))
    ax.imshow(out)
    ax.set_title(opencv_title)
    ax.axis("off")
    plt.show()

    return out

In [ ]:
def find_points(matcher, l_frame1, r_frame1, display=True):
    # convert to tensor
    # set first frame and first points for left and right
    #l_frame1_gray = cv2.cvtColor(l_frame1, cv2.COLOR_RGB2GRAY)
    l_frame1_gray_t = torch.from_numpy(l_frame1/255.).float()[None, None]

    #r_frame1_gray = cv2.cvtColor(r_frame1, cv2.COLOR_RGB2GRAY)
    r_frame1_gray_t = torch.from_numpy(r_frame1/255.).float()[None, None]
    
    # finding matching points in first frame for l and r
    pred = matcher({'image0': l_frame1_gray_t, 'image1': r_frame1_gray_t})

    # get sections from output (pred)
    l_kp = pred['keypoints0'][0].cpu().numpy()
    r_kp = pred['keypoints1'][0].cpu().numpy()
    matches = pred['matches0'][0].cpu().numpy()
    confidence = pred['matching_scores0'][0].detach().cpu().numpy()

    # keep only the valid matches and make points from found kp
    valid = matches > -1
    l_pts = l_kp[valid]
    r_pts = r_kp[matches[valid]]

    # plot kp and matches betwwen 2 images
    color = cm.jet(confidence[valid])
    text = ['SuperGlue',
        'Keypoints: {}:{}'.format(len(l_kp), len(r_kp)),
        'Matches: {}'.format(len(l_pts))]
    
    # view matches found
    if display:
        out = make_matching_plot_fast(
                    l_frame1, r_frame1, l_kp, r_kp, l_pts, r_pts, color, text,
                    path=None, show_keypoints=True)
    
    return l_frame1, r_frame1, l_pts, r_pts

In [ ]:
def find_object_point(n, nadd, l_pts, r_pts, l_frame1, r_frame1):
    # Create some random colors (100 of them)
    color = np.random.randint(0, 255, (100, 3))

    # copies for drawing on
    left_c = l_frame1.copy()
    right_c = r_frame1.copy()

    # left drawing
    for i in l_pts[n:n+nadd]:
        pointl = (i[0].astype(int), i[1].astype(int))
        left_c = cv2.circle(left_c, pointl, 2, (255,0,0), -1)

    # right drawing
    for i in r_pts[n:n+nadd]:
        pointr = (i[0].astype(int), i[1].astype(int))
        right_c = cv2.circle(right_c, pointr, 2, (255,0,0), -1)

    # plot left and right
    fig, ax = plt.subplots(figsize=(20,20))
    ax.imshow(np.hstack([left_c, right_c]))
    ax.set_title('Left -> Right')
    ax.axis("off")
    plt.show()
    
    return n, nadd

In [ ]:
def data_3D_revised(l_saved_point, r_saved_point, display='base'):
    # camera baselines
    b = 49.9465
    f = 1.93
    
    zdata = []
    xdata = []
    ydata = []
    
    for i in range(len(l_saved_point)):
        x1 = l_saved_point[i][0]
        x2 = r_saved_point[i][0]

        z = (b*f*190)/(np.sqrt((x1-x2)**2))
        
        zdata.append(z)
    
    # use only the left frame to find x and y over frames
    for i in range(len(l_saved_point)):
        x1 = l_saved_point[i][0]
        y1 = l_saved_point[i][1]
        
        # distance from 0 point
        x2 = 0
        y2 = 0
        
        z = zdata[i]
        
        x = ((x1-x2)/(f*210))*z
        y = ((y1-y2)/(f*210))*z
        
        xdata.append(x)
        ydata.append(y)
    
    return xdata, ydata, zdata

In [ ]:
def import_stereo_video(left_name, right_name, start_loss=20, end_loss=20, display=True, only_left=False):
    # load in video file
    left_vid = cv2.VideoCapture(left_name)
    right_vid = cv2.VideoCapture(right_name)
    
    # print some specs
    fps = right_vid.get(5)
    frame_count = right_vid.get(7)
    print("Loaded in video:\nFPS: {} \tFrame Count: {}".format(int(fps), int(frame_count)-end_loss-start_loss))
    
    # convert to arrays
    left_array = []
    ret = True
    while ret == True:
        ret, frame = left_vid.read()
        if frame is not None:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            left_array.append(frame)
    left_array = np.array(left_array)
    left_array = left_array[start_loss:-end_loss]
    print("Left Array Loaded  | Size: {}".format(left_array.shape))

    right_array = []
    if only_left == False:
        ret = True
        while ret == True:
            ret, frame = right_vid.read()
            if frame is not None:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                right_array.append(frame)
        right_array = np.array(right_array)
        right_array = right_array[start_loss:-end_loss]
        print("Right Array Loaded | Size: {}".format(left_array.shape))
    
    if display:
        plt.imshow(left_array[0])
        plt.title("First Frame:")
        plt.axis("off")
        plt.show()
    
    return left_array, right_array

## Load video in:

In [ ]:
# Camera rotation translation stuff
# camera matrix and distortion for 640x480 res: 
camera_matrix = np.array([[382.63, 0, 321.202],[0, 382.63, 239.855],[0, 0, 1]])
distortion =  np.array([-0.055629, 0.063853, -0.000203, -0.000965, -0.020792])

In [ ]:
# load in video array:
left_name = 'stabilization_videos/vid3.mp4'
right_name = 'stabilization_videos/vid3-right.mp4'
left_array, right_array = import_stereo_video(left_name, right_name, start_loss=50, end_loss=50, display=True, only_left=False)

# Finding transforms:

## KLT Method:

In [ ]:
# finding transforms with KLT 
transforms = np.zeros((len(left_array)-1, 3), np.float32)

for i in range(len(left_array)-1):
    # completion tracker
    clear_output(wait=True)
    print("{}/{} - {}%".format(i+2, len(left_array), np.round((i*100)/(len(left_array)-2), 1)))
    
    frameg = cv2.cvtColor(left_array[i], cv2.COLOR_RGB2GRAY)
    prev_pts = cv2.goodFeaturesToTrack(frameg,
                                     maxCorners=200,
                                     qualityLevel=0.01,
                                     minDistance=30,
                                     blockSize=3)

    # Calculate optical flow (i.e. track feature points)
    curr_pts, status, err = cv2.calcOpticalFlowPyrLK(left_array[i], left_array[i+1], prev_pts, None)
    
    # Sanity check
    assert prev_pts.shape == curr_pts.shape
    
    # Filter only valid points
    idx = np.where(status==1)[0]
    
    prev_pts = prev_pts[idx]
    curr_pts = curr_pts[idx]
    
    m, _ = cv2.estimateAffinePartial2D(prev_pts, curr_pts)
    
    # extract info from matrix
    dx = m[0,2]
    dy = m[1,2]
    da = np.arctan2(m[1,0], m[0,0])

    # create transform matrix
    transforms[i] = [dx,dy,da]

## Super Method:

In [ ]:
# use superpoint and superglue to find matches between frames
config = {
        'superpoint': {
            'nms_radius': 2,
            'keypoint_threshold': 0.1,  #0
            'max_keypoints': -1
        },
        'superglue': {
            'weights': 'indoor',
            'sinkhorn_iterations': 20,
            'match_threshold': 0.2,  # 0.01
        }
    }

# initializing matcher
matcher = Matching(config).eval()#.to(device)

In [ ]:
# finding transforms with superpoint/glue
transforms = np.zeros((len(left_array)-1, 3), np.float32)

for i in range(len(left_array)-1):
    # completion tracker
    clear_output(wait=True)
    print("{}/{} - {}%".format(i+1, len(left_array)-1, np.round((i*100)/(len(left_array)-2), 1)))
    
    # use super point and super glue to find good points to track between frames
    frame1, frame2, pts1, pts2 = find_points(matcher, left_array[i], left_array[i+1], display=False)
    # can use a feature detection and then use KLT to find the same points in the next frame, I beleive my method is just as effective if not more
    # however this method takes considerably longer to compute so probably not a smart option

    m, _ = cv2.estimateAffinePartial2D(pts1, pts2)
    
    # extract info from matrix
    dx = m[0,2]
    dy = m[1,2]
    da = np.arctan2(m[1,0], m[0,0])

    # create transform matrix
    transforms[i] = [dx,dy,da]

# Finding difference:

## smoothing path method:

In [ ]:
# calculate trajectory, smooth out this trajectory
# use cumulative sum 
trajectory = np.cumsum(transforms, axis=0)

In [ ]:
# can plot the trajectories
def plot_trajectory(trajectory):
    fig, ax = plt.subplots(1,3, figsize=(20,5))

    ax[0].plot(trajectory[:,0])
    ax[0].set_title("x")
    ax[0].set_ylabel("x (pixels)")
    ax[0].set_xlabel("Frames")
    ax[1].plot(trajectory[:,1])
    ax[1].set_title("y")
    ax[1].set_ylabel("y (pixels)")
    ax[1].set_xlabel("Frames")
    ax[2].plot(trajectory[:,2])
    ax[2].set_title("rotation")
    ax[2].set_ylabel("y (pixels)")
    ax[2].set_xlabel("Frames")

    plt.show()

In [ ]:
# we now have 3 curves for the trajectory over time, now we need to smooth them using moving average filter
def movingAverage(curve, radius):
    window_size = 2*radius + 1 # both ways plus the point itseld
    # filter
    f = np.ones(window_size)/window_size
    # Add padding to the boundaries
    curve_pad = np.lib.pad(curve, (radius, radius), 'edge')
    # Apply convolution
    curve_smoothed = np.convolve(curve_pad, f, mode='same')
    # Remove padding
    curve_smoothed = curve_smoothed[radius:-radius]
    return curve_smoothed

In [ ]:
def smooth(trajectory, radius):
    smoothed_trajectory = np.copy(trajectory)
    # filter the x, y and angle curves
    for i in range(3):
        smoothed_trajectory[:,i] = movingAverage(trajectory[:,i], radius=radius)
    return smoothed_trajectory

In [ ]:
smoothed_trajectory = smooth(trajectory, 30)

In [ ]:
plot_trajectory(trajectory)
plot_trajectory(smoothed_trajectory)

In [ ]:
# calculate difference between smoothed and normal trajectory
difference = smoothed_trajectory - trajectory

## zero movement method:

In [ ]:
# calculate trajectory, zero out this trajectory
# use cumulative sum 
trajectory = np.cumsum(transforms, axis=0)

In [ ]:
# using a difference to 0, smooth trajectory should be all at 0 movement.

In [ ]:
difference = np.zeros_like(trajectory) - trajectory

In [ ]:
plot_trajectory(transforms)

# Stabilizing:

In [ ]:
# add the difference to the transforms to get smoothed transformation between frames
transforms_smooth = transforms + difference

In [ ]:
stabilized_array = []

# apply the smoothed transformation to every frame to get stabilized video
for i in range(len(left_array)-1):
    
    dx = transforms_smooth[i,0]
    dy = transforms_smooth[i,1]
    da = transforms_smooth[i,2]
    
    # reconstruct the transformation matrix
    m = np.zeros((2,3), np.float32)
    m[0,0] = np.cos(da)
    m[0,1] = -np.sin(da)
    m[1,0] = np.sin(da)
    m[1,1] = np.cos(da)
    m[0,2] = dx
    m[1,2] = dy
    
    # apply affine warping to frames
    frame_stabilized = cv2.warpAffine(left_array[i], m, (2720, 1530))      #(640,480)

    # save to array
    stabilized_array.append(frame_stabilized)

In [ ]:
# save stabilized array (convert to video on desktop)
np.save("vid3.npy", stabilized_array)

# Checking Stabilization (point tracking):

In [ ]:
# start with array
# find points
# track point using klt over video
# plot those points and see movement
# compare movement of normal to stabilized

In [ ]:
array = stabilized_array

In [ ]:
# set frame 1 and 2 and find points in frame 1 to track
frame1 = array[0]
frame1g = cv2.cvtColor(frame1, cv2.COLOR_RGB2GRAY)
frame1gt = torch.from_numpy(frame1g/255.).float()[None, None]

frame2 = array[1]
frame2g = cv2.cvtColor(frame2, cv2.COLOR_RGB2GRAY)
frame2gt = torch.from_numpy(frame2g/255.).float()[None, None]

# use superpoint and superglue to find some points:
config = {
        'superpoint': {
            'nms_radius': 2,
            'keypoint_threshold': 0.1,  #0
            'max_keypoints': -1
        },
        'superglue': {
            'weights': 'indoor',
            'sinkhorn_iterations': 20,
            'match_threshold': 0.02,  # 0.01
        }
    }

# initializing matcher
matcher = Matching(config).eval()#.to(device)

# finding matching points in first frame and second
pred = matcher({'image0': frame1gt, 'image1': frame2gt})

# get sections from output (pred)
kp1 = pred['keypoints0'][0].cpu().numpy()
kp2 = pred['keypoints1'][0].cpu().numpy()
matches = pred['matches0'][0].cpu().numpy()
confidence = pred['matching_scores0'][0].detach().cpu().numpy()

# keep only the valid matches and make points from found kp
valid = matches > -1
pts1 = kp1[valid]
pts2 = kp2[matches[valid]]

In [ ]:
def find_object_point(n, nadd, pts1, frame1):
    # Create some random colors (100 of them)
    color = np.random.randint(0, 255, (100, 3))

    # copies for drawing on
    frame1c = frame1.copy()

    # left drawing
    for i in pts1[n:n+nadd]:
        point = (i[0].astype(int), i[1].astype(int))
        frame1c = cv2.circle(frame1c, point, 3, (0,255,0), -1)

    # plot left and right
    fig, ax = plt.subplots(figsize=(20,20))
    ax.imshow(frame1c)
    ax.set_title('Left -> Right')
    ax.axis("off")
    plt.show()
    
    return n, nadd

In [ ]:
n, nadd = find_object_point(100, 1, pts1, array[0])

In [ ]:
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# use points in frame 1 found using superpoint
prev_pts = pts1[n:n+nadd]

points = [prev_pts]

for i in range(len(array)-1):
    # completion tracker
    clear_output(wait=True)
    print("{}/{} - {}%".format(i+2, len(left_array), np.round((i*100)/(len(left_array)-2), 1)))

    # Calculate optical flow (i.e. track feature points)
    curr_pts, status, err = cv2.calcOpticalFlowPyrLK(left_array[i], left_array[i+1], prev_pts, None, **lk_params)
    
    # Sanity check
    assert prev_pts.shape == curr_pts.shape
    
    # Filter only valid points
    idx = np.where(status==1)[0]
    
    prev_pts = prev_pts[idx]
    curr_pts = curr_pts[idx]
    
    # save points 
    points.append(curr_pts)
    
    # set prev to current points for next iteration
    prev_pts = curr_pts

In [ ]:
points = np.array(points)
points = points.reshape(points.shape[0],2)

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(25,5))

#ax[0].plot(point_beam[:, 0]-point_beam[0][0])
ax[0].plot(points[:, 0]-points[0][0])
ax[0].set_title("X")

#ax[1].plot(point_beam[:, 1]-point_beam[0][1])
ax[1].plot(points[:, 1]-points[0][1])
ax[1].set_title("Y")

#ax[2].scatter(point_beam[:, 0]-point_beam[0][0], point_beam[:, 1]-point_beam[0][1])
ax[2].plot(points[:, 0]-points[0][0], points[:, 1]-points[0][1])
ax[2].set_title("X vs Y")

plt.show()